In [ ]:
!pip install mecab

# import

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time, datetime, MeCab, collections, os

# chromeDriver

In [34]:
#chromeをheadlessで起動
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver",  chrome_options=options)
#driver = webdriver.Chrome("/usr/local/bin/chromedriver")
top_page = "https://news.yahoo.co.jp/"
dt_now = datetime.datetime.now()
driver.get(top_page)

<ipython-input-34-a990e741f5d2>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/local/bin/chromedriver",  chrome_options=options)


# ニュースリストの取得

In [37]:
df = pd.DataFrame(columns=['news_title','comment_url'])

#topicの選択
select_M_Topic = driver.find_element_by_link_text("トップ").click()
time.sleep(0.1)
select_S_Topic = driver.find_element_by_link_text("主要").click()
time.sleep(0.1)

#ニュース記事への移動
topPage = driver.current_url
#news[0].click()
for a in range(8):
#     print(a+1)
    section = driver.find_element_by_tag_name("section")
    news_list = section.find_elements_by_tag_name("li")
    mv_news = news_list[a].click()
    time.sleep(0.1)
    news_main = driver.find_element_by_tag_name("article")
    news_title = news_main.find_elements_by_tag_name("p")
    print(news_title[2].text)

    #コメントページへの移動
    try:
        news_comment_url = driver.find_element_by_class_name("news-comment-plugin").get_attribute("data-full-page-url")
#         print(news_comment_url)
        df.loc[a] = [news_title[2].text,news_comment_url]    
    except:
#         print('This news page have no comment.')
        df.loc[a] = [news_title[2].text,'This news page have no comment.']    
    finally:
        driver.get(topPage) 
        #time.sleep(1)

dt = dt_now.strftime('%Y%m%d')
df.to_csv('./NewsList/'+str(dt)+'.csv')

中継 西村担当相と尾身会長が臨時会見
「完全離脱」カウントダウン　貿易交渉、なお溝　英EU
トヨタ自動車元町工場の火事は鎮火　開発棟内の車両から出火か　生産ラインに影響なし
三陽商会が正社員の早期退職募集、人数定めず　保養所も売却
東急線20年ぶり新駅は「新綱島駅」。東急新横浜線'22年度下期開業
西武、松坂と来季1000万円減の2000万円で契約締結…「少しでも恩返しできるように」
宮崎大輔、離婚成立を認めた！ “暴行逮捕”騒動の13歳下恋人と「今も一緒に住んでます」
林家こん平さん死去　７７歳「笑点」で人気「１、２、３、チャラ～ン」


# コメント情報の取得

In [41]:
dt = dt_now.strftime('%Y%m%d')
df = pd.read_csv('./NewsList/'+str(dt)+'.csv')
for i in range(df.shape[0]):
    title = df['news_title'][i]
    print(title)
    try:    
        url = df['comment_url'][i]
        driver.get(url)
    except:
        print('No_Comments.')
        continue
    else:
        print('Have a Comments.')
        break
print(title)

#コメント情報の取得

comment_boxes = []

#df = pd.DataFrame(columns=['number','日付','ユーザー名','コメント','good','bad'])
df = pd.DataFrame(columns=['number','日付','ユーザー名','コメント'])
start = 1
end = 200
base_url = driver.current_url
page_url = '?page='
order_url = '&order=older'

iframe = driver.find_element_by_class_name("news-comment-plguin-iframe")
driver.switch_to_frame(iframe)
end = driver.find_element_by_class_name("pagenation")
pages = end.find_elements_by_class_name("rapidnofollow")
last_page = pages[7].text
driver.switch_to_default_content()
print(last_page)
for page in range(start,int(last_page)):
    
    driver.get(base_url+page_url+str(page)+order_url) 
    print(driver.current_url)
#    time.sleep(1)
    iframe = driver.find_element_by_class_name("news-comment-plguin-iframe")
    driver.switch_to_frame(iframe)
    
    comment_boxes = driver.find_elements_by_class_name("root")
    page_comment =0
    
    for comment_box in comment_boxes:
        

        
        page_comment +=1
        comment_number = str(page)+'-'+str(page_comment)
        
        #ユーザー名取得
        elem_name = comment_box.find_element_by_class_name("rapidnofollow")
        name = elem_name.text
        #names.append(name)
        
        #日付取得
        elem_date = comment_box.find_element_by_class_name("date")
        date = elem_date.text
        #dates.append(date) 
        
        #コメント取得
        try:
            elem_comment = comment_box.find_element_by_class_name("cmtBody")
        except:
            elem_comment = comment_box.find_element_by_class_name("yjxComment")

        comment = elem_comment.text
        comment.strip("\n")
        #comments.append(comment)
        
        #good数取得
        try:
            agree_box = comment_box.find_element_by_class_name("good")
            elem_agree = agree_box.find_element_by_class_name("userNum")
            agree = elem_agree.text
        except:
            pass
        #agrees.append(agree)

        #bad数取得
        try:
            disagree_box = comment_box.find_element_by_class_name("bad")
            elem_disagree = disagree_box.find_element_by_class_name("userNum")
            disagree = elem_disagree.text
        except:
            pass
        #disagrees.append(disagree)
        #df.loc[(page-1)*10+page_comment] = [comment_number,date, name, comment, agree, disagree]
        df.loc[(page-1)*10+page_comment] = [comment_number,date, name, comment]


df.to_csv('./NewsComment/'+title+".csv")



中継 西村担当相と尾身会長が臨時会見
No_Comments.
「完全離脱」カウントダウン　貿易交渉、なお溝　英EU
Have a Comments.
「完全離脱」カウントダウン　貿易交渉、なお溝　英EU


<ipython-input-41-b1db154b7553>:30: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame(iframe)


IndexError: list index out of range

In [111]:
df = pd.read_csv('yahoo_news_comment.csv')

wakati = MeCab.Tagger("-Owakati")
chasen = MeCab.Tagger("-Ochasen")
comment_list = df['コメント']
for a in range(0,df.shape[0]):
    print(a)
#    all_comment += str(comment_list[a])
#print(all_comment)
# words = wakati.parse(all_comment).split()
# word_count = collections.Counter(words)
# print(word_count)

FileNotFoundError: [Errno 2] No such file or directory: 'yahoo_news_comment.csv'

In [33]:
driver.quit()

In [40]:
df = pd.read_csv('./NewsList/'+str(dt)+'.csv')
df.shape[0]
df['news_title'][1]

'「完全離脱」カウントダウン\u3000貿易交渉、なお溝\u3000英EU'